In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
beer_data = cPickle.load(open('../../beer_data/beer_vec_ds_df10.p', 'rb'))

Using TensorFlow backend.


[]


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
train_X = beer_data.X[train_idxs]
X_tf = np.zeros((train_X.shape[0], beer_data.vocab_size))
for i in range(len(train_X)) :
    X_tf[i, train_X[i, :]] = 1.

X_tf = X_tf[:, 2:]
train_Xtf = X_tf

val_X = beer_data.X[val_idxs]
X_tf = np.zeros((val_X.shape[0], beer_data.vocab_size))
for i in range(len(val_X)) :
    X_tf[i, val_X[i, :]] = 1.

X_tf = X_tf[:, 2:]
val_Xtf = X_tf

In [4]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras import optimizers

learning_rate = 5e-5
batch_size = 64
vocab_size = beer_data.vocab_size - 2
intermediate_dim = 500
latent_dim = 200
epochs = 1000
epsilon_std = 1.0
activation = 'tanh'

x = Input(shape=(vocab_size,), name='x')
h = Dense(intermediate_dim, activation=activation, name='h')(x)
mu = Dense(latent_dim, name='mu')(h)
log_sigma2 = Dense(latent_dim, name='l')(h)
encoder = Model(x, mu)

# reparameterized sampler for normal distributions
def sample_norm(args):
    '''reparameterized sampling from normal distribution'''
    mu, log_var = args
    epsilon = K.random_normal(shape=(K.shape(mu)[0], latent_dim,), mean=0.)
    return mu + K.exp(0.5 * log_var) * epsilon

# decoder / generative network
z = Lambda(sample_norm, output_shape=(latent_dim,), name='z')([mu, log_sigma2])
e = Dense(vocab_size, name='e')(z)

def log_softmax(x, axis=None):
    x0 = x - K.max(x, axis=axis, keepdims=True)
    log_sum_exp_x0 = K.log(K.sum(K.exp(x0), axis=axis, keepdims=True))
    return x0 - log_sum_exp_x0

def kl_loss(x, e): 
    return (- 0.5 * K.sum(1 + log_sigma2 - K.square(mu) - K.exp(log_sigma2), axis=-1))


def cross_ent_loss(x, e): 
    return - K.sum(x * log_softmax(e, axis=-1), axis=-1) 
    

def vae_loss(x, e):
    xent_loss = cross_ent_loss(x, e)
    kld = kl_loss(x, e)
    return xent_loss + kld


opt = optimizers.adam(lr=learning_rate)
vae = Model(x, e)
vae.compile(optimizer=opt, 
            loss=vae_loss)

In [ ]:
from keras import callbacks
patience = 0
earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='min')

vae.fit(train_Xtf,  
        train_Xtf, 
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1, 
        callbacks=[earlyStopping], 
        validation_split=0.1)

Train on 73872 samples, validate on 8208 samples
Epoch 1/1000
73872/73872 [==============================] - 29s - loss: 652.9218 - val_loss: 648.7383
Epoch 2/1000
73872/73872 [==============================] - 29s - loss: 648.9754 - val_loss: 645.4938
Epoch 3/1000
73872/73872 [==============================] - 29s - loss: 646.3455 - val_loss: 643.4037
Epoch 4/1000
73872/73872 [==============================] - 29s - loss: 644.5708 - val_loss: 641.7129
Epoch 5/1000
73872/73872 [==============================] - 29s - loss: 643.2042 - val_loss: 640.8848
Epoch 6/1000
73872/73872 [==============================] - 28s - loss: 642.1464 - val_loss: 639.8315
Epoch 7/1000
73872/73872 [==============================] - 29s - loss: 641.3772 - val_loss: 639.0850
Epoch 8/1000
73872/73872 [==============================] - 29s - loss: 640.6602 - val_loss: 638.4774
Epoch 9/1000
73872/73872 [==============================] - 29s - loss: 639.9163 - val_loss: 637.8398
Epoch 10/1000
73872/73872 [======

In [6]:
embedds = encoder.predict(val_Xtf)

In [7]:
idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [8]:
from sklearn.preprocessing import normalize
embedds_n = normalize(embedds, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [25]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.69504547126
0 0.679372734004
3 0.698324965887
2 0.695540049732


In [9]:
from sklearn.metrics import roc_auc_score
scores = np.array(scores)
for aspect_j in H :
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print aspect_j, np.mean(aucs)

1 0.64926456748
0 0.644489786921
3 0.651441195122
2 0.650329906976
